# Lectura al MongoDB

El objetivo de este Jupyter notebook es: i) verificar que la base de datos MongoDB está correctamente instalada y podemos hacer una lectura correcta de los datos; y ii) Extraer la tablas mencionadas en el punto 4 y guardar los datos en formato .csv.

En este notebook ejecutaremos los siguientes test:
1. Conexión a la base de datos MongoDB en local.
2. Revisar el tamaño y la última actualización de los datos.
3. Crear una muestra pequeña de datos para confirmar que los comandos funcionan.
4. Crear las tablas solicitadas en el diagrama ER con una muestra de los datos.

In [2]:
# Importa las librerías
from pymongo import MongoClient
import pandas as pd
import time

## 1. Conexión a la base de datos MongoDB en local.

Antes de ejecutar este notebook acuérdate de arrancar la base de datos MongoDB. Se recomienda utilizar Mongo Compass para una gestión más intuitiva de la base de datos.

In [3]:
start_time = time.time()

# Reemplaza 'localhost' con la dirección de tu servidor MongoDB si es diferente
# y '27017' con el puerto correspondiente si has configurado uno distinto.
client = MongoClient('mongodb://localhost:27017/')

#Nombre las bases de datos
#db = client['Contratos_EDCA']
db = client['Contratos_EDCA']

# Obtén una colección para verificar la conexión
try:
    # Reemplaza 'Contratos_EDCA' con el nombre de una colección existente en tu base de datos
    colecciones = db.list_collection_names()  # Lista todas las colecciones para verificar la conexión
    if 'Contratos_EDCA_Bulk' in colecciones:
        print("Conexión exitosa a la base de datos y la colección encontrada.")
    else:
        print("Conexión exitosa a la base de datos, pero la colección no fue encontrada.")
except Exception as e:
    print(f"Error al conectar a la base de datos: {e}")


# Listar todas las bases de datos
databases = client.list_database_names()
collections = db.list_collection_names()

# Imprimir la lista de bases de datos
print("Bases de datos disponibles son:", databases)

print("\n")

print("Colecciones dentro de la base de datos:", collections)

elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")

Conexión exitosa a la base de datos y la colección encontrada.
Bases de datos disponibles son: ['Contratos_EDCA', 'admin', 'config', 'local']


Colecciones dentro de la base de datos: ['Contratos_EDCA_Bulk']
Tiempo de ejecución: 0.06062626838684082 segundos


## 2. Revisar el tamaño y la última actualización de los datos.

La base de datos Contrataciones tiene dos colecciones: 
- Contratos_EDCA: 7 GB
- Sample_Contratos_EDCA: 800 MB

In [4]:
start_time = time.time()
## NO CORRER ESTE CHUNK SI NO ES NECESARIO ##

# Imprimir el tamaño de la base de datos, número de colecciones y última actualización

# Obteniendo el tamaño de la base de datos en GB
print(f"Obteniendo el tamaño de la base de datos.")
db_stats = db.command("dbstats")
db_size_gb = db_stats['storageSize'] / (1024**3)  # Convertir bytes a GB
print(f"Tamaño de la base de datos: {db_size_gb} GB")

# Obteniendo el número de documentos en la colección 'Contratos_EDCA'
##print(f"Obteniendo el número de documentos. Este proceso puede tardar hasta 3 minutos...")
num_documents = db['Contratos_EDCA_Bulk'].count_documents({})
print(f"Número de documentos en la colección 'Contratos_EDCA_Sample': {num_documents}")
elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")

Obteniendo el tamaño de la base de datos.
Tamaño de la base de datos: 6.616733551025391 GB
Número de documentos en la colección 'Contratos_EDCA_Sample': 2742267
Tiempo de ejecución: 72.76326489448547 segundos



## 4. Crear las tablas solicitadas en el diagrama ER con una muestra de los datos.

Extrae las siguientes tablas de la muestra de la base de datos:

- Participantes_Proveedores (Parties)
- Licitacion (Tender)
- asignacion (Awards)
- comprador (Parties + Buyer)
- documentos_tender (Tender)
- items_adq (Awards)
- items_tender (Tender)


### 4.1 Participantes_Proveedores (Origen: Parties)

In [15]:
# Iniciando lectura de tiempo de ejecución
start_time = time.time()

# Consulta actualizada a la base de datos Contratos_EDCA
#consulta_actualizada = db['Contratos_EDCA_Bulk'].find({"releases.buyer.id": "SESNA-330"}, {})
consulta_actualizada = db['Contratos_EDCA_Bulk'].find({}, {})
# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = []
for contrato in consulta_actualizada:
    for party in contrato.get('releases', [{}])[0].get('parties', []):
        if party:
            roles = party.get('roles', [])
            if "tenderer" in roles or all(role in roles for role in ["tenderer", "supplier"]):
                contrato_dict = {
                    'cve_expediente': contrato.get('releases', [{}])[0].get('tender', {}).get('id', ''),
                    'cve_contrato': contrato.get('releases', [{}])[0].get('awards', [{}])[0].get('id', ''),
                    'identifier_id': party.get('identifier', {}).get('id', ''),
                    'roles': roles,
                    'name': party.get('name', ''),
                    'identifier_legalName': party.get('identifier', {}).get('legalName', ''),
                    'identifier_scheme': party.get('identifier', {}).get('scheme', ''),
                    'identifier_uri': party.get('identifier', {}).get('uri', ''),
                    'address_countryName': party.get('address', {}).get('countryName', ''),
                    'address_locality': party.get('address', {}).get('locality', ''),
                    'address_postalCode': party.get('address', {}).get('postalCode', ''),
                    'address_region': party.get('address', {}).get('region', ''),
                    'address_streetAddress': party.get('address', {}).get('streetAddress', ''),
                    'contactPoint_email': party.get('contactPoint', {}).get('email', ''),
                    'contactPoint_name': party.get('contactPoint', {}).get('name', ''),
                    'contactPoint_telephone': party.get('contactPoint', {}).get('telephone', '')
                }
                datos.append(contrato_dict)

# Creando el dataframe de Participantes_Proveedores
df_participantes_proveedores = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv_files
df_participantes_proveedores.to_csv('../data/Processed/participantes_proveedores_v2.csv_files', index=False, encoding='utf-8')
print("Proceso terminado. \n     El dataset de Participantes_Proveedores tiene el siguiente tamaño: (filas x columnas)")
print(df_participantes_proveedores.shape)

#Terminando lectura de tiempo de ejecución e imprimiendola
elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")

Proceso terminado. 
     El dataset de Participantes_Proveedores tiene el siguiente tamaño: (filas x columnas)
(5623393, 16)
Tiempo de ejecución: 1364.737802028656 segundos


### 4.2 Licitación (Origen: Tender)

In [4]:
# Iniciando lectura de tiempo de ejecución
start_time = time.time()

# Consulta actualizada a la base de datos Contratos_EDCA
#consulta_actualizada = db['Contratos_EDCA_Bulk'].find({"releases.buyer.id": "SESNA-330"}, {})
consulta_actualizada = db['Contratos_EDCA_Bulk'].find({}, {})

# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = []
for contrato in consulta_actualizada:
    contrato_dict = {
        'cve_expediente': '',
        'procurementMethod': '',
        'procurementMethod_rationale': '',
        'status': '',
        'title': '',
        'description': '',
        'has_enquiries': '',
        'number_tenderers': '', 
        'tender_start_date': '',
        'tender_end_date': '',
        'award_start_date': '',
        'award_end_date': '',
        'enquiry_start_date': '',
        'enquiry_end_date': '',
        'procuring_entity_id': '',
        'procuring_entity_name': '',
        'value_currency_tender': '',
        'value_amount_tender': '',
        'award_criteria': '',
        'framework_agreement': '',
        'framework_agreement_platform': '',
        'framework_agreement_title': '',
        'submission_method': '', 
    }
    try:
        tender = contrato.get('releases', [{}])[0].get('tender', {})
        contrato_dict['cve_expediente'] = tender.get('id', '')  # id
        contrato_dict['procurementMethod'] = tender.get('procurementMethod', '')
        contrato_dict['procurementMethod_rationale'] = tender.get('procurementMethodRationale', '')
        contrato_dict['status'] = tender.get('status', '')
        contrato_dict['title'] = tender.get('title', '')
        contrato_dict['description'] = tender.get('description', '')
        contrato_dict['has_enquiries'] = tender.get('hasEnquiries', '')
        contrato_dict['number_tenderers'] = len(tender.get('tenderers', []))
        contrato_dict['tender_start_date'] = tender.get('tenderPeriod', {}).get('startDate', '')
        contrato_dict['tender_end_date'] = tender.get('tenderPeriod', {}).get('endDate', '')
        contrato_dict['award_start_date'] = tender.get('awardPeriod', {}).get('startDate', '')
        contrato_dict['award_end_date'] = tender.get('awardPeriod', {}).get('endDate', '')
        contrato_dict['enquiry_start_date'] = tender.get('enquiryPeriod', {}).get('startDate', '')
        contrato_dict['enquiry_end_date'] = tender.get('enquiryPeriod', {}).get('endDate', '')
        contrato_dict['procuring_entity_id'] = tender.get('procuringEntity', {}).get('id', '')  
        contrato_dict['procuring_entity_name'] = tender.get('procuringEntity', {}).get('name', '') 
        contrato_dict['value_currency_tender'] = tender.get('value', {}).get('currency', '')
        contrato_dict['value_amount_tender'] = tender.get('value', {}).get('amount', '')
        contrato_dict['award_criteria'] = tender.get('awardCriteria', '')
        contrato_dict['framework_agreement'] = tender.get('frameworkAgreement', '')  
        contrato_dict['framework_agreement_platform'] = tender.get('frameworkAgreementPlatform', '')
        contrato_dict['framework_agreement_title'] = tender.get('frameworkAgreementTitle', '')
        contrato_dict['submission_method'] = tender.get('submissionMethod', '')[0]
           
    except AttributeError:
        pass
    datos.append(contrato_dict)

# Creando el dataframe de licitacion
df_licitacion = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv_files
df_licitacion.to_csv('../data/Processed/licitacion_sesna_data.csv_files', index=False, encoding='utf-8')
print("Proceso terminado. \n     El dataset de Licitación tiene el siguiente tamaño: (filas x columnas)")

print(df_licitacion.shape)

#Terminando lectura de tiempo de ejecución e imprimiendola
elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")

Proceso terminado. 
     El dataset de Licitación tiene el siguiente tamaño: (filas x columnas)
(2742267, 23)
Tiempo de ejecución: 1207.5515050888062 segundos


### 4.3 Asignación (awards)

In [3]:
import pandas as pd

# Iniciando lectura de tiempo de ejecución
start_time = time.time()

# Consulta actualizada a la base de datos Contratos_EDCA
#consulta_actualizada = db['Contratos_EDCA_Bulk'].find({"releases.buyer.id": "SESNA-330"}, {})
consulta_actualizada = db['Contratos_EDCA_Bulk'].find({}, {})

# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = []
for contrato in consulta_actualizada:
    for award in contrato.get('releases', [{}])[0].get('awards', []):
        contrato_dict = {
            'cve_expediente': contrato.get('releases', [{}])[0].get('tender', {}).get('id', ''),
            'cve_contrato': award.get('id', ''),
            'status': award.get('status', ''),
            'description_award': award.get('description', ''),
            'title_award': award.get('title', ''),
            'contract_start_date': award.get('contractPeriod', {}).get('startDate', ''),
            'contract_end_date': award.get('contractPeriod', {}).get('endDate', ''),
            'value_amount': award.get('value', {}).get('amount', ''),
            'value_currency': award.get('value', {}).get('currency', ''),
            'suppliers_id': award.get('suppliers', [{}])[0].get('id', ''),
            'suppliers_name': award.get('suppliers', [{}])[0].get('name', ''),
            'docs_url_awards': award.get('documents', [{}])[0].get('url', '') if award.get('documents') else '',
            'docs_title_awards': award.get('documents', [{}])[0].get('title', '') if award.get('documents') else '',
            'docs_language_awards': award.get('documents', [{}])[0].get('language', '') if award.get('documents') else '',
            'docs_id_awards': award.get('documents', [{}])[0].get('id', '') if award.get('documents') else '',
            'docs_format_awards': award.get('documents', [{}])[0].get('format', '') if award.get('documents') else '',
            'docs_type_awards': award.get('documents', [{}])[0].get('documentType', '') if award.get('documents') else '',
            'docs_descr_awards': award.get('documents', [{}])[0].get('description', '') if award.get('documents') else '',
            'docs_date_published_awards': award.get('documents', [{}])[0].get('datePublished', '') if award.get('documents') else ''
        }
        datos.append(contrato_dict)

# Creando el dataframe de Asignación
df_asignacion = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv_files
df_asignacion.to_csv('../data/Processed/asignacion_sesna_data.csv_files', index=False, encoding='utf-8')
print("Proceso terminado. \n     El dataset de Asignación tiene el siguiente tamaño: (filas x columnas)")
print(df_asignacion.shape)

#Terminando lectura de tiempo de ejecución e imprimiendola
elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")


Proceso terminado. 
     El dataset de Asignación tiene el siguiente tamaño: (filas x columnas)
(4163085, 19)
Tiempo de ejecución: 1251.3326761722565 segundos


### 4.4 Comprador (Origen: Parties + Buyer)

In [5]:
# Iniciando lectura de tiempo de ejecución
start_time = time.time()

# Consulta actualizada a la base de datos Contratos_EDCA
#consulta_actualizada = db['Contratos_EDCA_Bulk'].find({"releases.buyer.id": "SESNA-330"}, {})
consulta_actualizada = db['Contratos_EDCA_Bulk'].find({}, {})

# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = []
for contrato in consulta_actualizada:
    parties = contrato.get('releases', [{}])[0].get('parties', [])
    for party in parties:
        if party:  # Check if party is not None
            roles = party.get('roles', [])
            if 'buyer' in roles or 'procuringEntity' in roles:
                contrato_dict = {
                    'cve_expediente': contrato.get('releases', [{}])[0].get('tender', {}).get('id', ''),  # id
                    'cve_contrato': contrato.get('releases', [{}])[0].get('awards', [{}])[0].get('id', ''),
                    'identifier_id_inst': party.get('identifier', {}).get('id', ''),
                    'roles': roles[0] if roles else '',
                    'name': party.get('name', ''),
                    'identifier_legal_name': party.get('identifier', {}).get('legalName', ''),
                    'identifier_schema': party.get('identifier', {}).get('scheme', ''),
                    'identifier_uri': party.get('identifier', {}).get('uri', ''),
                    'addres_country_name': party.get('address', {}).get('countryName', ''),
                    'addres_locality': party.get('address', {}).get('locality', ''),
                    'address_postalcode': party.get('address', {}).get('postalCode', ''),
                    'address_region': party.get('address', {}).get('region', ''),
                    'addres_streetaddress': party.get('address', {}).get('streetAddress', ''),
                    'contact_point_email': party.get('contactPoint', {}).get('email', ''),
                    'contact_point_name': party.get('contactPoint', {}).get('name', ''),
                    'contact_point_telephone': party.get('contactPoint', {}).get('telephone', '')
                }
                datos.append(contrato_dict)

# Creando el dataframe de comprador
df_comprador = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv_files
df_comprador.to_csv('../data/Processed/comprador_sesna_data.csv_files', index=False, encoding='utf-8')
print("Proceso terminado. \n     El dataset de Comprador tiene el siguiente tamaño: (filas x columnas)")

print(df_comprador.shape)

#Terminando lectura de tiempo de ejecución e imprimiendola
elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")

Proceso terminado. 
     El dataset de Comprador tiene el siguiente tamaño: (filas x columnas)
(5484414, 16)
Tiempo de ejecución: 1336.5880494117737 segundos


### 4.5 Documentos_Tender (Tender)

In [4]:
import time
import pandas as pd

# Iniciando lectura de tiempo de ejecución
start_time = time.time()

# Consulta actualizada a la base de datos Contratos_EDCA
consulta_actualizada = db['Contratos_EDCA_Bulk'].find({}, {})

# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = []
for contrato in consulta_actualizada:
    # Accede a cada release. Asumimos que cada contrato tiene al menos un release.
    releases = contrato.get('releases', [])
    for release in releases:
        # Accede a los documentos dentro de 'tender', si existen
        tender = release.get('tender', {})
        documents = tender.get('documents', [])
        
        if documents:
            for document in documents:
                contrato_dict = {
                    'cve_expediente': tender.get('id', ''),
                    'docs_title_tender': document.get('title', ''),
                    'docs_type_tender': document.get('documentType', ''),
                    'docs_language_tender': document.get('language', ''),
                    'docs_date_published_tender': document.get('datePublished', ''),
                    'docs_id_tender': document.get('id', ''),
                    'docs_format_tender': document.get('format', ''),
                    'docs_description_tender': document.get('description', ''),
                    'docs_url_tender': document.get('url', '')
                }
                datos.append(contrato_dict)
        else:
            # Create a record with empty document fields if there are no documents
            contrato_dict = {
                'cve_expediente': tender.get('id', ''),
                'docs_title_tender': '',
                'docs_type_tender': '',
                'docs_language_tender': '',
                'docs_date_published_tender': '',
                'docs_id_tender': '',
                'docs_format_tender': '',
                'docs_description_tender': '',
                'docs_url_tender': ''
            }
            datos.append(contrato_dict)

# Creando el dataframe de documentos tender
df_documentos_tender = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv_files
df_documentos_tender.to_csv('../data/Processed/documentos_tender_sesna_data_V2.csv_files', index=False, encoding='utf-8')
print("Proceso terminado. \n     El dataset de Documentos Tender tiene el siguiente tamaño: (filas x columnas)")
print(df_documentos_tender.shape)

# Terminando lectura de tiempo de ejecución e imprimiéndola
elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")


Proceso terminado. 
     El dataset de Documentos Tender tiene el siguiente tamaño: (filas x columnas)
(3127507, 9)
Tiempo de ejecución: 1080.4137599468231 segundos


### 4.6 ITEMS_ADQ (Origen: Awards)

In [3]:
# Iniciando lectura de tiempo de ejecución
start_time = time.time()

# Consulta actualizada a la base de datos Contratos_EDCA
#consulta_actualizada = db['Contratos_EDCA_Bulk'].find({"releases.buyer.id": "SESNA-330"}, {})
consulta_actualizada = db['Contratos_EDCA_Bulk'].find({}, {})

# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = []
for contrato in consulta_actualizada:
    for award in contrato.get('releases', [{}])[0].get('awards', [{}]):
        for item in award.get('items', []):
            contrato_dict = {
                'cve_expediente': contrato.get('releases', [{}])[0].get('tender', {}).get('id', ''),  # id
                'cve_contrato': award.get('id', ''),
                'items_unit_val_currency_awards': item.get('unit', {}).get('value', {}).get('currency', ''),
                'items_unit_val_amount_awards': item.get('unit', {}).get('value', {}).get('amount', ''),
                'items_unit_name_awards': item.get('unit', {}).get('name', ''),
                'items_classion.uri': item.get('classification', {}).get('uri', ''),
                'items_classi_scheme_awards': item.get('classification', {}).get('scheme', ''),
                'items_class_id_awards': item.get('classification', {}).get('id', ''),
                'items_class_description_awards': item.get('classification', {}).get('description', ''),
                'items_quantity_awards': item.get('quantity', ''),
                'items_id_awards': item.get('id', ''),
                'items_description_awards': item.get('description', ''),
            }
            datos.append(contrato_dict)

# Creando el dataframe de ITEMS
df_items = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv_files
df_items.to_csv('../data/Processed/items_adq_sesna_data.csv_files', index=False, encoding='utf-8')
print("Proceso terminado. \n     El dataset de Items tiene el siguiente tamaño: (filas x columnas)")

print(df_items.shape)

#Terminando lectura de tiempo de ejecución e imprimiendola
elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")

Proceso terminado. 
     El dataset de Items tiene el siguiente tamaño: (filas x columnas)
(12179867, 12)
Tiempo de ejecución: 1883.3869903087616 segundos


### 4.7 ITEMS_TENDER (Origen: Tender)

In [5]:
# Iniciando lectura de tiempo de ejecución
start_time = time.time()

# Consulta actualizada a la base de datos Contratos_EDCA
consulta_actualizada = db['Contratos_EDCA_Bulk'].find({"releases.buyer.id": "SESNA-330"}, {})

consulta_actualizada = db['Contratos_EDCA_Bulk'].find({}, {})

# Creando una lista de diccionarios para facilitar la creación del dataframe
datos = []
for contrato in consulta_actualizada:
    for item in contrato.get('releases', [{}])[0].get('tender', {}).get('items', []):
        contrato_dict = {
            'cve_expediente': contrato.get('releases', [{}])[0].get('tender', {}).get('id', ''), 
            'items_unit_name_tender': item.get('unit', {}).get('name', ''),
            'items_class_id_tender': item.get('classification', {}).get('id', ''),
            'items_class_description_tender': item.get('classification', {}).get('description', ''),
            'items_quantity_tender': item.get('quantity', ''),
            'items_id_tender': item.get('id', ''),
            'items_description_tender': item.get('description', ''),
        }
        datos.append(contrato_dict)

# Creando el dataframe de Tender Items
df_tender_items = pd.DataFrame(datos)

# Exportando el dataframe a un archivo csv_files
df_tender_items.to_csv('../data/Processed/tender_items_sesna_data.csv_files', index=False, encoding='utf-8')
print("Proceso terminado. \n     El dataset de Tender Items tiene el siguiente tamaño: (filas x columnas)")

print(df_tender_items.shape)

#Terminando lectura de tiempo de ejecución e imprimiendola
elapsed_time = time.time() - start_time
print(f"Tiempo de ejecución: {elapsed_time} segundos")

Proceso terminado. 
     El dataset de Tender Items tiene el siguiente tamaño: (filas x columnas)
(11419377, 7)
Tiempo de ejecución: 1783.5181257724762 segundos
